# Example 3
In the examples so far, we have shown how to compute the traction maps with images in the ideal format. In Example 3 we will demonstrate how you can process data with different input dimensions. We also make use of an experimental feature that predicts the ROI.


Until the repo is made public you'll be required to enter a git token. This can be created by going to [GitHub](https://docs.github.com/en/github/authenticating-to-github/creating-a-personal-access-token) and only ticking the repo. Do not share this key. 

In [ ]:
from getpass import getpass
TOKEN = getpass('Enter the git token: ')

In [ ]:
!git clone https://$TOKEN@github.com/rg314/pytraction.git
!pip install pytraction/

In [ ]:
!pytraction_get_data

In [ ]:
import numpy as np
from skimage import io
import matplotlib.pyplot as plt

from pytraction.core import TractionForce
from pytraction.utils import plot

In [ ]:
pix_per_um = 1
E = 3000 # Young's modulus in Pa

img_path_bead = '../data/example3/Beads3.tif'
img_path_cell = '../data/example3/Cell3.tif'
ref_path = '../data/example3/BeadsStop.tif'

traction_obj = TractionForce(pix_per_um, E=E, segment=True)
img, ref, roi = traction_obj.load_data(img_path_bead, ref_path)

Notice that when we try to load the data we get a RuntimeWarning because the input shape is not correct.

In [ ]:
img_bead = io.imread(img_path_bead)
img_cell = io.imread(img_path_cell)
ref = io.imread(ref_path)

print(f'The shape of the bead image is {img_bead.shape}')
print(f'The shape of the cell image is {img_cell.shape}')
print(f'The shape of the reference image is {ref.shape}')

All the images have 11 dimentions which could be a stack in space or time. 

In [ ]:
# plot every 2 bead slices
fig, ax = plt.subplots(1,5, figsize=(16,16))
for idx, axi in enumerate(ax.ravel()):
    axi.set_title(f'beads slice {idx*2}')
    axi.imshow(img_bead[idx*2,:,:], vmax=42168)
    axi.set_axis_off()
plt.tight_layout()

# plot every 2 cell slices
fig, ax = plt.subplots(1,5, figsize=(16,16))
for idx, axi in enumerate(ax.ravel()):
    axi.set_title(f'img slice {idx*2}')
    axi.imshow(img_cell[idx*2,:,:], vmax=42168)
    axi.set_axis_off()
plt.tight_layout()

The intensity of both images decays as the number of slices increase and the beads become out of focus which suggests theses are z slices. Therefore, we should take the maximum Z projection of the stack in the 0th axis to get a (w,h) shaped image. We should then stack the bead and cell image together. 

In [ ]:
def z_project(img_path):
    img = io.imread(img_path)
    img_max= np.max(img, axis=0)
    return img_max

bead = z_project(img_path_bead)
cell = z_project(img_path_cell)
ref = z_project(ref_path)

img = np.stack([[bead, cell]])
ref = np.stack([ref, ref])

print(f'The shape of the bead image is {img.shape}')
print(f'The shape of the cell image is {ref.shape}')

The images now have the correct shapes. Notice that I used `np.stack([[bead, cell]])` for the `img` to ensure the shape is `(1, 2, 512, 512)`. Let's save the images to disk and reload them using `load_data`

In [ ]:
# save images to disk
io.imsave('../data/example3/tfm.tif', img)
io.imsave('../data/example3/tfm-ref.tif', ref)

In [ ]:
pix_per_um = 1
E = 3000 # Young's modulus in Pa

img_path = '../data/example3/tfm.tif'
ref_path = '../data/example3/tfm-ref.tif'

traction_obj = TractionForce(pix_per_um, E=E, segment=True)
img, ref, _ = traction_obj.load_data(img_path, ref_path)

This time no error message was returned. I also set `segment=True` which will have used an experimental feature to predict the ROI. Let's run the stack.

In [ ]:
log3 = traction_obj.process_stack(img, ref, verbose=0, crop=True)

In [ ]:
for frame in range(len(log3)):
    plot(log3, frame=frame, vmax=300, mask=False)

Note that no ROI was provided, and it was still possible to use the segment feature to locate the cell. If you are experiencing issues with this feature please raise an issue on the GitHub as this will allow us to update the feature.

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(16,16))
ax[0].imshow(img[0,1,:,:], cmap='gray')
ax[0].set_axis_off()
ax[0].set_title('Input image')

ax[1].imshow(log3['cell_roi'][0], cmap='gray')
ax[1].set_axis_off()
ax[1].set_title('Auto-segment image')